In [1]:
!pip install mlflow torchvision>=0.9.1 torch==1.9.0 pytorch-lightning==1.4.0

In [2]:
%cd /tmp

/tmp


In [3]:
!git clone https://github.com/mlflow/mlflow.git

fatal: destination path 'mlflow' already exists and is not an empty directory.


In [4]:
%cd mlflow/examples/pytorch/MNIST

/tmp/mlflow/examples/pytorch/MNIST


In [5]:
!mlflow run . --no-conda

2021/09/03 17:12:06 INFO mlflow.projects.utils: === Created directory /tmp/tmp6v5nz5hx for downloading remote URIs passed to arguments of type 'path' ===
2021/09/03 17:12:06 INFO mlflow.projects.backend.local: === Running command 'python mnist_autolog_example.py \
  --max_epochs 5 \
  --gpus 0 \
  --accelerator None \
  --batch_size 64 \
  --num_workers 3 \
  --lr 0.001 \
  --es_patience 3 \
  --es_mode min \
  --es_verbose True \
  --es_monitor val_loss
' in run with ID '605a74852ebe42f7acdfba013cfb9307' === 
/home/sa/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. 

/home/sa/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/pytorch_lightning/core/datamodule.py:424: LightningDeprecationWarning: DataModule.prepare_data has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.prepare_data.
  f"DataModule.{name} has already been called, so it will not be called again. "
Testing: 0it [00:00, ?it/s]/home/sa/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/deprecate/deprecation.py:115: LightningDeprecationWarning: The `accuracy` was deprecated since v1.3.0 in favor of `torchmetrics.functional.classification.accuracy.accuracy`. It will be removed in v1.5.0.
  stream(template_mgs % msg_args)
Testing:  92%|████████████████████████████▋  | 145/157 [00:00<00:00, 217.71it/s]--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'avg_test_acc': 0.9748208522796631}
--------------------------------------------------------------------

In [6]:
!tree -L 1 mlruns/0

mlruns/0
├── 4bb45ba9395b48feb5a76ded43e4f37d
├── 605a74852ebe42f7acdfba013cfb9307
├── 860c30a3e4b7436188128a5715ebbe80
└── meta.yaml

3 directories, 1 file


In [7]:
from torchvision import datasets

# load the training data
mnist_test = datasets.MNIST('/tmp/data', train=False, download=True)
mnist_test = list(mnist_test)[0]
img, category = mnist_test
display(img)
print(category)

/home/sa/miniconda3/envs/tempo-examples/lib/python3.7/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


7


In [8]:
import numpy as np
img_np = np.asarray(img).reshape((1, 28*28)).astype(np.float32)

In [9]:
import os
ARTIFACTS_FOLDER = os.path.join(
    os.getcwd(),
    "mlruns/0",
    os.listdir("mlruns/0")[0])
print(ARTIFACTS_FOLDER)

/tmp/mlflow/examples/pytorch/MNIST/mlruns/0/605a74852ebe42f7acdfba013cfb9307


In [10]:
from tempo.serve.metadata import ModelFramework
from tempo.serve.model import Model

mlflow_tag = "mlflow"

pytorch_mnist_model = Model(
    name="test-pytorch-mnist",
    platform=ModelFramework.MLFlow,
    local_folder=ARTIFACTS_FOLDER,
    uri="s3://tempo/basic/mnist",
    description="A pytorch MNIST model",
)



Insights Manager not initialised as empty URL provided.


In [11]:
from tempo.serve.loader import save
save(pytorch_mnist_model)

Solving environment: ...working... done

_openmp_mutex-4.5    | 22 KB     | ########## | 100% 
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
Installing pip dependencies: ...working... Ran pip subprocess with arguments:
['/home/sa/miniconda3/envs/tempo-593da01b-3783-41e1-a906-f8265bbc5269/bin/python', '-m', 'pip', 'install', '-U', '-r', '/tmp/condaenv.bg5q0sc7.requirements.txt']
Pip subprocess output:
  Using cached absl_py-0.13.0-py3-none-any.whl (132 kB)
  Using cached aiohttp-3.7.4.post0-cp37-cp37m-manylinux2014_x86_64.whl (1.3 MB)
  Using cached alabaster-0.7.12-py2.py3-none-any.whl (14 kB)
  Using cached alembic-1.4.1-py2.py3-none-any.whl
  Using cached alibi-0.5.7-py3-none-any.whl (307 kB)
  Using cached alibi_detect-0.6.0-py3-none-any.whl (143 kB)
  Using cached ansible-4.4.0-py3-none-any.whl
  Using cached ansible_core-2.11.4-py3-none-any.whl
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)

Pip subprocess error:
ERROR: Cannot install mlserver-mlflow==0.4.0 and mlserver_mlflow==0.4.1.dev1 because these package versions have conflicting dependencies.
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/user_guide/#fixing-conflicting-dependencies


CondaEnvException: Pip failed



CalledProcessError: Command 'conda env create --name tempo-593da01b-3783-41e1-a906-f8265bbc5269 --file /tmp/tmpu3zt4cjf.yml' returned non-zero exit status 1.

In [ ]:
from tempo import deploy_local
local_deployed_model = deploy_local(pytorch_mnist_model)

In [ ]:
local_prediction = local_deployed_model.predict(img_np)
print(np.nonzero(local_prediction.flatten() == 0))

In [ ]:
local_deployed_model.undeploy()

In [ ]:
%cd -0

In [ ]:
!kubectl apply -f k8s/rbac -n seldon

In [ ]:
from tempo.examples.minio import create_minio_rclone
import os
create_minio_rclone(os.getcwd()+"/rclone.conf")

In [ ]:
from tempo.serve.loader import upload
upload(pytorch_mnist_model)

In [ ]:
from tempo.serve.metadata import SeldonCoreOptions
runtime_options = SeldonCoreOptions(**{
        "remote_options": {
            "namespace": "seldon",
            "authSecretName": "minio-secret"
        }
    })

In [ ]:
from tempo import deploy_remote
remote_deployed_model = deploy_remote(pytorch_mnist_model, options=runtime_options)

In [ ]:
remote_prediction = remote_deployed_model.predict(img_np)
print(np.nonzero(remote_prediction.flatten() == 0))

In [ ]:
remote_deployed_model.undeploy()